In [90]:
!pip3 install langchain_experimental
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document

In [91]:
!pip3 install bs4

In [92]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import TokenTextSplitter
import bs4
load_dotenv()

raw_documents = WebBaseLoader('https://www.partselect.com/PS11752778-Whirlpool-WPW10321304-Refrigerator-Door-Shelf-Bin.htm?SourceCode=3&SearchTerm=PS11752778').load()
print("raw_documents: ", raw_documents)
text_splitter = TokenTextSplitter(chunk_size = 2048, chunk_overlap=24,model_name='gpt-4o')

documents = text_splitter.split_documents(raw_documents)
print(raw_documents)


llm = ChatOpenAI(temperature=0, model="gpt-4o", )
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(graph_documents)





raw_documents:  [Document(metadata={'source': 'https://www.partselect.com/PS11752778-Whirlpool-WPW10321304-Refrigerator-Door-Shelf-Bin.htm?SourceCode=3&SearchTerm=PS11752778', 'title': 'Official Whirlpool WPW10321304 Refrigerator Door Shelf Bin – PartSelect.com', 'description': 'OEM WPW10321304 - Refrigerator Door Shelf Bin - replacement. We offer authentic parts, and the expert advice you need to complete the repair. Same–day shipping and easy returns!', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOfficial Whirlpool WPW10321304 Refrigerator Door Shelf Bin – PartSelect.com\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\nSign In\n|\nCreate Account\n\n\n\n|\nYour Account\n\n\n\n\n\n\n\n Your Orders\r\n                    \n\n\n\n Your Subscriptions\r\n                    \n\n\n\n\nHome\n\nFind by Brand\n\nFind by Product\nFind by Symptom\nBlog\nRepair Help\nWater Filt

In [93]:
lmm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=[
        'Label','Part', 'Model', 'Manufacturer', 'Brand', 'InstallationInstruction', 
        'Symptom', 'Video', 'Section', 'Manual', 'Story', 'QnA', 'ProductType'
    ],  # Include all node types relevant to your model and part data
    allowed_relationships=[
        'COMPATIBLE_WITH', 'MANUFACTURED_BY', 'HAS_SECTION', 'HAS_PART', 
        'HAS_INSTALLATION_INSTRUCTION', 'HAS_SYMPTOM', 'HAS_VIDEO', 
        'HAS_MANUAL', 'HAS_STORY', 'HAS_QNA', 'WORKS_WITH_PRODUCT_TYPE', 
        'FIXES_SYMPTOM', 'USES_PART', 'USES_FIXING_PART', 'BRAND_DESTINATION',
        'RELATED_TO', 'REPLACES'
    ],  # Include all relevant relationships between nodes
)

graph_documents_filtered = lmm_transformer_filtered.convert_to_graph_documents(documents)
print(graph_documents_filtered)

[GraphDocument(nodes=[Node(id='Wpw10321304', type='Part'), Node(id='Whirlpool', type='Manufacturer'), Node(id='Kenmore', type='Brand'), Node(id='Maytag', type='Brand'), Node(id='Kitchenaid', type='Brand'), Node(id='Refrigerator', type='Producttype'), Node(id='Door Won’T Open Or Close', type='Symptom'), Node(id='Ice Maker Won’T Dispense Ice', type='Symptom'), Node(id='Leaking', type='Symptom')], relationships=[Relationship(source=Node(id='Wpw10321304', type='Part'), target=Node(id='Whirlpool', type='Manufacturer'), type='MANUFACTURED_BY'), Relationship(source=Node(id='Wpw10321304', type='Part'), target=Node(id='Kenmore', type='Brand'), type='BRAND_DESTINATION'), Relationship(source=Node(id='Wpw10321304', type='Part'), target=Node(id='Maytag', type='Brand'), type='BRAND_DESTINATION'), Relationship(source=Node(id='Wpw10321304', type='Part'), target=Node(id='Kitchenaid', type='Brand'), type='BRAND_DESTINATION'), Relationship(source=Node(id='Wpw10321304', type='Part'), target=Node(id='Refri

In [94]:
!pip3 install py2neo

In [95]:

from langchain.graphs import Neo4jGraph
NEO4J_URI ='neo4j://localhost:7687'
NEO4J_USERNAME = 'neo4j'
NEO4J_PASSWORD = 'password'

graph = Neo4jGraph()


In [96]:
for node in graph_documents_filtered[0].nodes:
    print(node)

id='Wpw10321304' type='Part'
id='Whirlpool' type='Manufacturer'
id='Kenmore' type='Brand'
id='Maytag' type='Brand'
id='Kitchenaid' type='Brand'
id='Refrigerator' type='Producttype'
id='Door Won’T Open Or Close' type='Symptom'
id='Ice Maker Won’T Dispense Ice' type='Symptom'
id='Leaking' type='Symptom'


In [97]:
for relationship in graph_documents_filtered[0].relationships:
    print(relationship)


source=Node(id='Wpw10321304', type='Part') target=Node(id='Whirlpool', type='Manufacturer') type='MANUFACTURED_BY'
source=Node(id='Wpw10321304', type='Part') target=Node(id='Kenmore', type='Brand') type='BRAND_DESTINATION'
source=Node(id='Wpw10321304', type='Part') target=Node(id='Maytag', type='Brand') type='BRAND_DESTINATION'
source=Node(id='Wpw10321304', type='Part') target=Node(id='Kitchenaid', type='Brand') type='BRAND_DESTINATION'
source=Node(id='Wpw10321304', type='Part') target=Node(id='Refrigerator', type='Producttype') type='WORKS_WITH_PRODUCT_TYPE'
source=Node(id='Wpw10321304', type='Part') target=Node(id='Door Won’T Open Or Close', type='Symptom') type='FIXES_SYMPTOM'
source=Node(id='Wpw10321304', type='Part') target=Node(id='Ice Maker Won’T Dispense Ice', type='Symptom') type='FIXES_SYMPTOM'
source=Node(id='Wpw10321304', type='Part') target=Node(id='Leaking', type='Symptom') type='FIXES_SYMPTOM'


In [100]:
print(graph_documents_filtered)

[{'nodes': [{'id': 'Wpw10321304', 'type': 'Part'}, {'id': 'Whirlpool', 'type': 'Manufacturer'}, {'id': 'Kenmore', 'type': 'Brand'}, {'id': 'Maytag', 'type': 'Brand'}, {'id': 'Kitchenaid', 'type': 'Brand'}, {'id': 'Refrigerator', 'type': 'Producttype'}, {'id': 'Door Won’T Open Or Close', 'type': 'Symptom'}, {'id': 'Ice Maker Won’T Dispense Ice', 'type': 'Symptom'}, {'id': 'Leaking', 'type': 'Symptom'}], 'relationships': [{'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Whirlpool', 'type': 'Manufacturer'}, 'type': 'MANUFACTURED_BY'}, {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Kenmore', 'type': 'Brand'}, 'type': 'BRAND_DESTINATION'}, {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Maytag', 'type': 'Brand'}, 'type': 'BRAND_DESTINATION'}, {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Kitchenaid', 'type': 'Brand'}, 'type': 'BRAND_DESTINATION'}, {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'i

In [98]:
graph.add_graph_documents(graph_documents_filtered)

In [99]:
from neo4j import GraphDatabase

# Setup your Neo4j connection
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

# Function to create nodes
def create_node(tx, node_id, node_type):
    query = """
    MERGE (n:NodeType {id: $id, type: $type})
    RETURN n
    """
    tx.run(query, id=node_id, type=node_type)

# Function to create relationships
def create_relationship(tx, source_id, source_type, target_id, target_type, relationship_type):
    query = """
    MATCH (a:NodeType {id: $source_id, type: $source_type})
    MATCH (b:NodeType {id: $target_id, type: $target_type})
    MERGE (a)-[r:REL_TYPE {type: $relationship_type}]->(b)
    RETURN r
    """
    tx.run(query, source_id=source_id, source_type=source_type,
           target_id=target_id, target_type=target_type,
           relationship_type=relationship_type)

# Function to process each GraphDocument (now accessing keys of dictionary)
def add_graph_data(graph_documents_filtered):
    with driver.session() as session:
        for graph_document in graph_documents_filtered:  # Process each GraphDocument in the list
            # Insert nodes
            for node in graph_document['nodes']:  # Access 'nodes' key directly
                session.write_transaction(create_node, node['id'], node['type'])

            # Insert relationships
            for relationship in graph_document['relationships']:  # Access 'relationships' key directly
                session.write_transaction(
                    create_relationship,
                    relationship['source']['id'], relationship['source']['type'],
                    relationship['target']['id'], relationship['target']['type'],
                    relationship['type']
                )

# Example of graph_documents_filtered data structure (using dictionary format)
graph_documents_filtered = [
    {
        'nodes': [
            {'id': 'Wpw10321304', 'type': 'Part'},
            {'id': 'Whirlpool', 'type': 'Manufacturer'},
            {'id': 'Kenmore', 'type': 'Brand'},
            {'id': 'Maytag', 'type': 'Brand'},
            {'id': 'Kitchenaid', 'type': 'Brand'},
            {'id': 'Refrigerator', 'type': 'Producttype'},
            {'id': 'Door Won’T Open Or Close', 'type': 'Symptom'},
            {'id': 'Ice Maker Won’T Dispense Ice', 'type': 'Symptom'},
            {'id': 'Leaking', 'type': 'Symptom'}
        ],
        'relationships': [
            {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Whirlpool', 'type': 'Manufacturer'}, 'type': 'MANUFACTURED_BY'},
            {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Kenmore', 'type': 'Brand'}, 'type': 'BRAND_DESTINATION'},
            {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Maytag', 'type': 'Brand'}, 'type': 'BRAND_DESTINATION'},
            {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Kitchenaid', 'type': 'Brand'}, 'type': 'BRAND_DESTINATION'},
            {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Refrigerator', 'type': 'Producttype'}, 'type': 'WORKS_WITH_PRODUCT_TYPE'},
            {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Door Won’T Open Or Close', 'type': 'Symptom'}, 'type': 'FIXES_SYMPTOM'},
            {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Ice Maker Won’T Dispense Ice', 'type': 'Symptom'}, 'type': 'FIXES_SYMPTOM'},
            {'source': {'id': 'Wpw10321304', 'type': 'Part'}, 'target': {'id': 'Leaking', 'type': 'Symptom'}, 'type': 'FIXES_SYMPTOM'}
        ]
    }
]

# Call the function to add data into Neo4j
add_graph_data(graph_documents_filtered)

# Close the driver connection after use
driver.close()


/var/folders/_s/49hfhkwd75n_f9vl_c9csh680000gn/T/ipykernel_52282/3113512900.py:35: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_node, node['id'], node['type'])
/var/folders/_s/49hfhkwd75n_f9vl_c9csh680000gn/T/ipykernel_52282/3113512900.py:39: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(
